In [1]:
import pandas as pd
import numpy as np
import bcrypt
from sqlalchemy import delete
from tqdm import tqdm
import emoji

from src.pingers import ping_ads_insights_all_platforms, ping_shops, ping_creative, ping_raw_creative_and_target, ping_creative_and_performance
from src.database.session import db
from src.s3 import read_csv_from_s3
from src.models import *
from src.schemas import *
from src.crud import *
from src.utils.database import row_to_dict
from src.utils.hash_password import hash_password
from src.scripts.saving_s3_files.save_global_descriptive_statistics_to_s3 import save_global_descriptive_statistics_to_s3
from src.crud.google.google_daily_performance import ga_daily_performance
from src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3 import save_google_descriptive_statistics_to_s3

2023-02-27 13:28:13.717 | DEBUG    | src.database.session:<module>:19 - base: lebesgue-dev.cgviinyjedbs.eu-central-1.rds.amazonaws.com


base I am using: lebesgue-dev.cgviinyjedbs.eu-central-1.rds.amazonaws.com


In [2]:
df = save_google_descriptive_statistics_to_s3(db=db)

2023-02-27 13:28:14.034 | DEBUG    | src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3:save_google_descriptive_statistics_to_s3:45 - pinging shops...
2023-02-27 13:28:21.288 | DEBUG    | src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3:save_google_descriptive_statistics_to_s3:48 - done!
2023-02-27 13:28:22.748 | DEBUG    | src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3:save_google_descriptive_statistics_to_s3:68 - we have 4 done shop ids.
  0%|          | 2/716 [00:04<24:49,  2.09s/it]2023-02-27 13:28:26.777 | DEBUG    | src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3:save_google_descriptive_statistics_to_s3:86 - shop_id: 43980593
2023-02-27 13:28:27.266 | DEBUG    | src.scripts.saving_s3_files.save_google_descriptive_statistics_to_s3:save_google_descriptive_statistics_to_s3:110 - Skipam jer nemam ['spend_USD']
  1%|          | 5/716 [00:06<13:06,  1.11s/it]2023-02-27 13:28:29.067 | DEBUG    | src.scripts

save_google_descriptive_statistics_to_s3: 0:51:00.137720


In [4]:
df.reset_index().groupby(["year_month", "feature"]).sum()

,,count_ads,spend_USD,impr,link_clicks,purch,count_ads_by_shop,spend_USD_by_shop,impr_by_shop,link_clicks_by_shop,purch_by_shop,count_shops
year_month,feature,,,,,,,,,,,
2019-03,type,8.0,439.397227,35092.0,1484.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-04,type,7.0,30.283303,4008.0,138.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-05,type,8.0,51.645570,8682.0,240.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-06,type,4.0,183.050030,15622.0,1000.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-07,type,2.0,225.822785,34804.0,1774.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-08,type,2.0,45.015069,16728.0,408.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-09,type,2.0,59.795057,42050.0,266.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-10,type,9.0,1278.747231,129440.0,14560.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0
2019-11,type,3.0,174.104882,15964.0,962.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


In [2]:
query = ga_daily_performance.query_performance(db=db, shop_id=16038, monthly=False)
df = ga_daily_performance.ping_performance(db=db, shop_id=16038)

In [3]:
df

,ad_id,shop_id,account_id,impr,link_clicks,purch,spend,purch_value,year_month,currency,type,ctr,cr,roas,spend_USD
0,78922099836,16038,2717281159,1108,10,0,4.300000,0.000000,2015-11,USD,DISPLAY_STANDARD,0.009025,0.000000,0.000000,4.300000
1,144973850634,16038,2717281159,1000,54,0,94.980000,0.000000,2016-10,USD,SEARCH_STANDARD,0.054000,0.000000,0.000000,94.980000
2,144973850634,16038,2717281159,3298,146,0,258.080000,0.000000,2016-11,USD,SEARCH_STANDARD,0.044269,0.000000,0.000000,258.080000
3,248635044211,16038,2717281159,470,218,12,57.180000,2331.320000,2018-02,USD,SEARCH_STANDARD,0.463830,0.055046,40.771598,57.180000
4,248635044211,16038,2717281159,222,84,4,25.980000,670.200000,2018-03,USD,SEARCH_STANDARD,0.378378,0.047619,25.796767,25.980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,629696717888,16038,2717281159,1908,1322,78,155.009390,15989.118079,2022-10,USD,SEARCH_STANDARD,0.692872,0.059664,103.149352,155.009390
1584,629696717888,16038,2717281159,7064,4844,234,725.083404,54668.316808,2022-11,USD,SEARCH_STANDARD,0.685730,0.048497,75.395901,725.083404
1585,629696717888,16038,2717281159,5548,3780,187,437.971594,30918.469876,2022-12,USD,SEARCH_STANDARD,0.681327,0.049491,70.594692,437.971594
1586,629696717888,16038,2717281159,8602,6080,234,534.716822,42743.015920,2023-01,USD,SEARCH_STANDARD,0.706812,0.038512,79.935798,534.716822


In [5]:
str(query)

'SELECT google_daily_performance.ad_id AS google_daily_performance_ad_id, google_daily_performance.shop_id AS google_daily_performance_shop_id, google_daily_performance.account_id AS google_daily_performance_account_id, sum(google_daily_performance.impressions) AS impr, sum(google_daily_performance.clicks) AS link_clicks, sum(google_daily_performance.conversions_purchase) AS purch, sum(google_daily_performance.spend) AS spend, sum(google_daily_performance.conversions_value_purchase) AS purch_value, google_ad_account.currency AS google_ad_account_currency, google_adgroup.type AS google_adgroup_type \nFROM google_daily_performance JOIN google_ad_account ON google_daily_performance.account_id = google_ad_account.google_id JOIN google_adgroup ON google_daily_performance.account_id = google_adgroup.account_id AND google_daily_performance.adgroup_id = google_ad_account.google_id AND google_daily_performance.shop_id = google_ad_account.shop_id \nWHERE google_daily_performance.shop_id IN (__[P

In [3]:
df

NameError: name 'df' is not defined

In [30]:
from src.models.facebook.ad_creative_features import AdCreativeFeatures
from sqlalchemy.orm import Session
import emoji
from collections import Counter


def get_emoji_distribution_by_shop(db: Session, shop_id: int, text_position: str = 'primary')->dict:
    """

    Args:
        text_position (str, optional): One of 'primary', 'title' and 'description'. Defaults to 'primary'.

    Returns:
        dict: Dictionary of the form {emoji: frequency} e.g. {'✨': 993, '☝️': 12, '🌸': 22}
    """

    if text_position not in ["primary", "description", "title"]:
        raise ValueError(f'{text_position}. Argument text_position must be either primary, title or description.')
    query = db.query(AdCreativeFeatures.value).filter(AdCreativeFeatures.shop_id==shop_id, AdCreativeFeatures.feature==text_position).all()
    text = " ".join([" ".join(a[0]) for a in query])
    emojis = [x['emoji'] for x in emoji.emoji_list(text)]
    emojis
    return dict(Counter(emojis))

In [32]:
d = get_emoji_distribution_by_shop(db=db, shop_id=16038)
d

{'✨': 993,
 '☝️': 12,
 '🌸': 22,
 '❤️': 50,
 '💓': 9,
 '🎵': 4,
 '💞': 3,
 '😉': 1,
 '💚': 4,
 '😍': 3,
 '👂': 1,
 '🤰': 1,
 '🎶': 1,
 '♥': 3,
 '🛍️': 1,
 '👀': 2,
 '👗': 1,
 '🕊️': 2,
 '💗': 1,
 '💛': 1,
 '💙': 1,
 '🤩': 1}

In [4]:
df = pd.DataFrame({'a': [0, 1], 'b': [2, 3], 'c': [4, 5]})
df.set_index(['a', 'b'], inplace=True)
df

,,c
a,b,
0,2,4
1,3,5


In [6]:
df.index.names

FrozenList(['a', 'b'])

In [2]:
table = save_global_descriptive_statistics_to_s3(db=db, force_from_scratch=True)

data/global/global_descriptive_statistics_2023-02-17.csv
['development/data/global/global_descriptive_statistics_2023-02-17.csv']


  0%|          | 0/687 [00:00<?, ?it/s]

shop_id: 37259196


2023-02-17 13:05:06.720 | DEBUG    | src.scripts.saving_s3_files.save_global_descriptive_statistics_to_s3:save_global_descriptive_statistics_to_s3:140 - full:                                      count_ads  spend_USD    impr  \
year_month feature    feature_value                                 
2022-06    emojis_any False                  1     648.97   16407   
                      True                   1     126.56    2434   
2022-07    emojis_any False                 17    6035.21  193541   
                      True                  18     850.03   26612   
2022-08    emojis_any False                 40   15778.12  688794   
                      True                  58    3385.90  125478   
2022-09    emojis_any False                  2    2143.39   89383   
2022-10    emojis_any False                  1   20115.17  729097   
2022-11    emojis_any False                  1    8235.99  308045   

                                     link_clicks  purch  count_ads_by_shop  \
yea

shop_id: 39679905


...

2023-02-17 13:05:39.508 | DEBUG    | src.scripts.saving_s3_files.save_global_descriptive_statistics_to_s3:save_global_descriptive_statistics_to_s3:140 - full:                                      count_ads   spend_USD   impr  \
year_month feature    feature_value                                 
2021-02    emojis_any True                   3    1.748323    112   
2022-04    emojis_any False                 10  326.844887  40023   
                      True                   1   17.869486   1960   
2022-05    emojis_any False                  4  220.573287  29328   
                      True                   3   64.779427   6552   
2022-06    emojis_any False                  2   45.242936   4629   
                      True                   3  388.900183  38193   
2022-07    emojis_any True                   3   74.293556   9511   
2022-08    emojis_any True                   6  272.768855  23349   
2022-09    emojis_any True                   2  156.779833  17108   

            

shop_id: 10943493


2023-02-17 13:05:44.312 | DEBUG    | src.scripts.saving_s3_files.save_global_descriptive_statistics_to_s3:save_global_descriptive_statistics_to_s3:140 - full:                                      count_ads  spend_USD     impr  \
year_month feature    feature_value                                  
2019-04    emojis_any False                  1     200.00    36490   
2019-05    emojis_any False                  4     111.00    15322   
                      True                   1      28.48      761   
2019-06    emojis_any False                 11     328.69    57501   
2019-07    emojis_any False                  3      56.68    10144   
2019-08    emojis_any False                  2     294.72    77899   
2019-09    emojis_any False                  5     112.58    24955   
2019-10    emojis_any False                  4     320.37    40152   
2019-11    emojis_any False                  9    4047.33   693676   
                      True                   1      67.64    11842   
2

shop_id: 9527262


KeyboardInterrupt: 

In [3]:
table

count_ads  spend_USD      impr  \
year_month feature    feature_value                                   
2017-01    emojis_any False                  3     200.55     26589   
                      True                  15    5332.47    670686   
2017-02    emojis_any False                  4     250.25     59971   
                      True                  17   11995.32    833008   
2017-03    emojis_any False                  4     335.29    161007   
...                                        ...        ...       ...   
2021-07    emojis_any True                  74  119440.18   7905390   
2021-08    emojis_any False                126  137307.52  10877670   
                      True                  82  127985.20   7742280   
2022-06    emojis_any True                   8   23813.46   1056248   
2022-07    emojis_any True                   5   44944.12   3892834   

                                     link_clicks  purch  count_ads_by_shop  \
year_month feature    feature_value                                          
2017-01    emojis_any False                  363      0           0.166667   
                      True                 12259     27           0.833333   
2017-02    emojis_any False                  297      2           0.190476   
                      True                 20067     13           0.809524   
2017-03    emojis_any False                  130      1           0.500000   
...                                          ...    ...                ...   
2021-07    emojis_any True                 24598   1352           0.230530   
2021-08    emojis_any False                24336   2152           0.605769   
                      True                 26588   1812           0.394231   
2022-06    emojis_any True                 10678      0           1.000000   
2022-07    emojis_any True                  9538      0           1.000000   

                                     spend_USD_by_shop  impr_by_shop  \
year_month feature    feature_value                                    
2017-01    emojis_any False                   0.036246      0.038133   
                      True                    0.963754      0.961867   
2017-02    emojis_any False                   0.020436      0.067158   
                      True                    0.979564      0.932842   
2017-03    emojis_any False                   0.017617      0.115786   
...                                                ...           ...   
2021-07    emojis_any True                    0.316732      0.327773   
2021-08    emojis_any False                   0.517570      0.584194   
                      True                    0.482430      0.415806   
2022-06    emojis_any True                    1.000000      1.000000   
2022-07    emojis_any True                    1.000000      1.000000   

                                     link_clicks_by_shop  purch_by_shop  \
year_month feature    feature_value                                       
2017-01    emojis_any False                     0.028759       0.000000   
                      True                      0.971241       1.000000   
2017-02    emojis_any False                     0.014585       0.133333   
                      True                      0.985415       0.866667   
2017-03    emojis_any False                     0.015323       0.043478   
...                                                  ...            ...   
2021-07    emojis_any True                      0.357166       0.262219   
2021-08    emojis_any False                     0.477889       0.542886   
                      True                      0.522111       0.457114   
2022-06    emojis_any True                      1.000000            NaN   
2022-07    emojis_any True                      1.000000            NaN   

                                     count_shops  
year_month feature    feature_value               
2017-01    emojis_any False                    1  
                      True      

In [2]:
shop_ids_query = db.query(FacebookAdsetInsights.shop_id).distinct()
shop_ids = pd.read_sql(shop_ids_query.statement, db.bind)["shop_id"].tolist()
chunk_size = 3

for i, start_idx in tqdm(enumerate(range(0, len(shop_ids) + 1, chunk_size)), total = len(shop_ids) // chunk_size):
    chunk = shop_ids[start_idx : start_idx + chunk_size]
    q = crud_fb_adset_insights.query_budget_split_by_target(db=db, shop_id=chunk)
    new_df = pd.read_sql(q.statement, db.bind)
    if i == 0:
        df = new_df
    else:
        df = pd.concat([df, new_df], axis=0).reset_index(drop=True)
    df.to_csv('data/budget_split_by_target.csv')

  1%|          | 3/338 [01:48<2:42:14, 29.06s/it]

In [3]:
df

,shop_id,spend_USD
0,16038,2202022.77
1,16038,211804.45


In [3]:
from sqlalchemy.dialects import postgresql
print(q.statement.compile(dialect=postgresql.dialect()))

SELECT anon_1.shop_id, sum(anon_1.spend / anon_2.rate_from_usd) AS "spend_USD" 
FROM (SELECT facebook_adset_insights.shop_id AS shop_id, facebook_adset_insights.account_id AS account_id, facebook_adset_insights.adset_id AS adset_id, facebook_adset.target AS target, sum(facebook_adset_insights.spend) AS spend 
FROM facebook_adset_insights JOIN facebook_adset ON facebook_adset_insights.shop_id = facebook_adset.shop_id AND facebook_adset_insights.account_id = facebook_adset.account_id AND facebook_adset_insights.adset_id = facebook_adset.adset_id 
WHERE facebook_adset_insights.shop_id IN (__[POSTCOMPILE_shop_id_1]) GROUP BY facebook_adset_insights.shop_id, facebook_adset_insights.account_id, facebook_adset_insights.adset_id, facebook_adset.target) AS anon_1 JOIN facebook_ad_account ON anon_1.account_id = facebook_ad_account.facebook_id AND anon_1.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_excha

In [5]:
stmt = (
    delete(FacebookAdsetInsights).
    where(FacebookAdsetInsights.shop_id == 5)
)

db.execute(stmt)

In [7]:
str(stmt)

'DELETE FROM facebook_adset_insights WHERE facebook_adset_insights.shop_id = :shop_id_1'

In [2]:
crud_fb_adset_insights.rebuild_table(db=db)

2023-02-17 10:17:33.237 | DEBUG    | src.crud.facebook.facebook_adset_insights:rebuild_table:25 - facebook_adset_insights


OperationalError: (psycopg2.OperationalError) could not translate host name "lebesgue-dev.cgviinyjedbs.eu-central-1.rds.amazonaws.com" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [7]:
list(map(list, np.array_split([1], 7)))

[[1], [], [], [], [], [], []]

In [6]:
df = pd.read_sql(db.query(FacebookDailyPerformance.shop_id).distinct().limit(5).statement, db.bind)

In [7]:
df

,shop_id
0,2
1,96
2,15873
3,15887
4,15892


In [11]:
df = ping_raw_creative_and_target(db=db, shop_id=268714, ad_id=6026939154285)

TypeError: ping_raw_creative_and_target() got an unexpected keyword argument 'db'

In [2]:
df = ping_creative(db=db, shop_id=268714)

In [4]:
df.loc[df.primary.isna(), 'ad_id']

1        6033996437485
2        6046729558485
3        6047215813285
4        6047215996285
5        6047216934685
             ...      
10763    6074009688853
10764    6074868893653
10765    6074982158453
10766    6102999128853
10767    6271846481853
Name: ad_id, Length: 1938, dtype: object

In [3]:
df.primary.count() / len(df)

0.8200222882615156

In [7]:
df.loc[df.primary.isna(), 'ad_id']

0        6026939154285
1        6033996437485
2        6046729558485
3        6047215813285
4        6047215996285
             ...      
10763    6074009688853
10764    6074868893653
10765    6074982158453
10766    6102999128853
10767    6271846481853
Name: ad_id, Length: 4695, dtype: object

In [3]:
df.primary.count() / len(df)

0.5639858841010401

In [3]:
df.primary.count() / len(df)

AttributeError: 'DataFrame' object has no attribute 'primary'

In [10]:
df[df.ad_id == '23842601978410353']

,shop_id,account_id,ad_id,creative_id,aov,industry,age_range,creative_type,cta_any,description,discounts_any,emojis,emojis_any,fact_words_any,free_shipping_any,hashtags_any,links_any,number_of_countries,number_of_custom_audiences,percentages_any,prices_any,primary,starts_with_question_any,target,targets_US,targets_english,title,urgency_any,user_addressing_any
0,268714,10159096477130290,23842601978410353,23842601978070353,36.687399,Home and garden,40,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
primaries = df.loc[df.primary.apply(len) > 0, 'primary'].apply(lambda x: x[0]).tolist()

TypeError: object of type 'float' has no len()

In [5]:
query = db.query(FacebookAdset.targeting).limit(1).all()

In [9]:
[a[0] for a in query]

[{'age_max': 65,
  'age_min': 21,
  'genders': [2],
  'geo_locations': {'regions': [{'key': '3845',
     'name': 'Arizona',
     'country': 'US'},
    {'key': '3847', 'name': 'California', 'country': 'US'},
    {'key': '3848', 'name': 'Colorado', 'country': 'US'},
    {'key': '3849', 'name': 'Connecticut', 'country': 'US'},
    {'key': '3852', 'name': 'Florida', 'country': 'US'},
    {'key': '3853', 'name': 'Georgia', 'country': 'US'},
    {'key': '3855', 'name': 'Idaho', 'country': 'US'},
    {'key': '3857', 'name': 'Indiana', 'country': 'US'},
    {'key': '3859', 'name': 'Kansas', 'country': 'US'},
    {'key': '3861', 'name': 'Louisiana', 'country': 'US'},
    {'key': '3862', 'name': 'Maine', 'country': 'US'},
    {'key': '3863', 'name': 'Maryland', 'country': 'US'},
    {'key': '3864', 'name': 'Massachusetts', 'country': 'US'},
    {'key': '3868', 'name': 'Missouri', 'country': 'US'},
    {'key': '3870', 'name': 'Nebraska', 'country': 'US'},
    {'key': '3871', 'name': 'Nevada', 'co

In [2]:
shop_id_query = db.query(FacebookDailyPerformance.shop_id).filter(FacebookDailyPerformance.date_start >= '2022-01-01').distinct()
shop_ids = pd.read_sql(shop_id_query.statement, db.bind)

shop_ids = shop_ids['shop_id'].tolist()
chunk_size = 3
for i, shop_ids_chunk in enumerate(np.array_split(shop_ids, len(shop_ids) // chunk_size)):
    print(shop_ids_chunk.tolist())
    query = crud_fb_daily_performance.query_budget_split_by_target(db=db, shop_id=shop_ids_chunk.tolist(), start_date='2022-01-01')
    print(query)
    new_df = pd.read_sql(query.statement, db.bind)
    if i == 0:
        df = new_df
    else:
        df = pd.concat([df, new_df], axis=0)
    if i == 2:
        break

[2, 96, 15873]
SELECT facebook_daily_performance.shop_id AS facebook_daily_performance_shop_id, facebook_adset.target AS facebook_adset_target, sum(facebook_daily_performance.spend / anon_1.rate_from_usd) AS spend 
FROM facebook_daily_performance JOIN facebook_adset ON facebook_daily_performance.shop_id = facebook_adset.shop_id AND facebook_daily_performance.account_id = facebook_adset.account_id AND facebook_daily_performance.adset_id = facebook_adset.adset_id JOIN facebook_ad_account ON facebook_daily_performance.account_id = facebook_ad_account.facebook_id AND facebook_daily_performance.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON facebook_ad_account.currency = anon_1.code 
WHERE facebook_daily_performance.shop_id IN (__[POSTCOMPILE_s

In [6]:
crud_streamlit_user.add_user(db=db, username='lebeg', hashed_password=hash_password('lebeg123.'), is_superuser=True)

In [3]:
def get_username_config():
    user_data = db.query(StreamlitUser).all()
    list_of_dicts = [row.__dict__ for row in user_data]
    print(list_of_dicts)
    result = {}
    for dic in list_of_dicts:
        username = dic["username"]
        dic["name"] = username
        dic.pop("username")
        dic["password"] = dic["hashed_password"]
        result[username] = dic
    return {"usernames": result}

In [2]:
user_data = db.query(StreamlitUser).all()

In [2]:
config = get_username_config()

NameError: name 'get_username_config' is not defined

In [7]:
ni = config['usernames']['ni']

In [21]:
ni['password'].encode()

b'\\x243262243132243946636435412f7538575334715a4b31484646734e6548633166647042467735737339794b6c774a49422e30335770487337504d69'

In [4]:
password = 'vorkosigan'
hashed = hash_password(password)
hashed

b'$2b$12$9Fcd5A/u8WS4qZK1HFFsNeMRZvkOyVEJD.20nJMZS7wH/jklUxulm'

In [5]:
type(hashed)

bytes

In [23]:
bcrypt.checkpw(password.encode(), ni["password"].encode())

ValueError: Invalid salt

In [2]:
q = db.query(FacebookAd).limit(5)

In [3]:
[type(a) for a in q]

[src.models.facebook.facebook_ad.FacebookAd,
 src.models.facebook.facebook_ad.FacebookAd,
 src.models.facebook.facebook_ad.FacebookAd,
 src.models.facebook.facebook_ad.FacebookAd,
 src.models.facebook.facebook_ad.FacebookAd]

In [2]:
crud_streamlit_user_shop.add_relationship(db=db, streamlit_user_id=19, shop_id=16038)

In [12]:
query = db.query(StreamlitUser)

In [13]:
df = pd.read_sql(query.statement, db.bind)

In [14]:
df

,id,username,hashed_password,is_superuser
0,3,lebeg,lebeg123.,True
1,17,123,\x243262243132243946636435412f7538575334715a4b...,False
2,16,k,\x243262243132243946636435412f7538575334715a4b...,False


In [25]:
q = db.query(StreamlitUser.username).all()
[a.username for a in q]

In [30]:
[a.username for a in q]

['lebeg', 'kikiriki', 'kikirik', 'kikirikajaja', 'kik', 'k', '123']

In [19]:
import bcrypt
import os
from dotenv import load_dotenv
load_dotenv()
from src.utils.hash_password import hash_password

In [17]:
os.getenv('STREAMLIT_SALT').encode()

b'$2b$12$9Fcd5A/u8WS4qZK1HFFsNe'

In [2]:
crud_streamlit_user.delete_user(db=db, username='lebega')

In [7]:
row = StreamlitUser(**dict(username='lebegaca', hashed_password='lebeg123.', is_superuser=True))
db.query(StreamlitUser).filter(StreamlitUser.username == 'lebegaca').delete()
db.commit()

In [33]:
query = crud_fb_daily_performance.query_budget_split_by_target(db=db, shop_id=[16038, 2], start_date='2022-01-01')
str(query)

'SELECT facebook_daily_performance.shop_id AS facebook_daily_performance_shop_id, facebook_adset.target AS facebook_adset_target, sum(facebook_daily_performance.spend / anon_1.rate_from_usd) AS spend \nFROM facebook_daily_performance JOIN facebook_adset ON facebook_daily_performance.shop_id = facebook_adset.shop_id AND facebook_daily_performance.account_id = facebook_adset.account_id AND facebook_daily_performance.adset_id = facebook_adset.adset_id JOIN facebook_ad_account ON facebook_daily_performance.account_id = facebook_ad_account.facebook_id AND facebook_daily_performance.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd \nFROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON facebook_ad_account.currency = anon_1.code \nWHERE facebook_daily_performance.shop_id IN (__[POSTCOMPILE_shop_id_1]) 

In [34]:
df = pd.read_sql(query.statement, db.bind)
df

,shop_id,target,spend
0,2,Target.acquisition,5159.64
1,2,Target.remarketing,221.28
2,16038,Target.acquisition,1690709.21
3,16038,Target.remarketing,167237.34


In [2]:
shops = ping_shops(db=db)

2023-02-09 14:46:01.495 | DEBUG    | src.pingers.ping_shops:ping_shops:23 - Here I am!


ping_shops: 0:00:19.410317


In [6]:
shop_id_query = db.query(FacebookDailyPerformance.shop_id).filter(FacebookDailyPerformance.date_start >= '2022-01-01').distinct()
shop_ids = pd.read_sql(shop_id_query.statement, db.bind)

shop_ids = shop_ids['shop_id'].tolist()
chunk_size = 3
for i, shop_ids_chunk in enumerate(np.array_split(shop_ids, len(shop_ids) // chunk_size)):
    print(shop_ids_chunk.tolist())
    query = crud_fb_daily_performance.query_budget_split_by_target(db=db, shop_id=shop_ids_chunk.tolist(), start_date='2022-01-01')
    print(query)
    new_df = pd.read_sql(query.statement, db.bind)
    if i == 0:
        df = new_df
    else:
        df = pd.concat([df, new_df], axis=0)
    if i == 2:
        break

[2, 96, 15873, 16038]
SELECT facebook_daily_performance.shop_id AS facebook_daily_performance_shop_id, facebook_adset.target AS facebook_adset_target, sum(facebook_daily_performance.spend / anon_1.rate_from_usd) AS spend 
FROM facebook_daily_performance JOIN facebook_adset ON facebook_daily_performance.shop_id = facebook_adset.shop_id AND facebook_daily_performance.account_id = facebook_adset.account_id AND facebook_daily_performance.adset_id = facebook_adset.adset_id JOIN facebook_ad_account ON facebook_daily_performance.account_id = facebook_ad_account.facebook_id AND facebook_daily_performance.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON facebook_ad_account.currency = anon_1.code 
WHERE facebook_daily_performance.shop_id IN (__[POSTCO

[array([36833384, 46864466, 43739609, 50592831, 36370064, 53002252,
        39579188, 31971959, 43985342, 45844908, 38489951, 20347698,
        68534913, 12153503, 33391601, 36143108, 12109463, 43242730,
        41490637, 13597214, 48425409, 31064070, 15332300, 13781927,
        43352241,   256652,  1370565,  2114125, 17807133, 51022062,
        31839270, 38580607, 37140876, 44270769,  1407292, 41224141,
        50604700, 49063542, 45008008, 76522436, 50254205, 31904124,
        21313648, 13131861, 41174878, 50783576, 17045701, 40954863,
           15887, 40330230, 22926764,  9596262, 45038286,  8573833,
        44512322, 38092292, 45836491, 39270994, 17506226, 56579596,
        38833446, 74399413, 41363055, 38962439, 47140968,  8112778,
        44086229, 45045056, 46360217, 38894179, 18675218, 30704322,
        46317801, 41273036, 43954850, 12140410, 45764574,  9231199,
        25963515, 31623603, 40281047,    16006, 39329571, 27227973,
         9893977, 49074615, 41020843, 58264127, 

In [11]:
def f(df):
    print(df)
    df.spend = df.spend / df.spend.sum()
    return df

In [12]:
df.groupby('shop_id').transform(f)

0    Target.acquisition
1    Target.remarketing
Name: target, dtype: object


AttributeError: 'Series' object has no attribute 'spend'

In [ ]:
import pandas as pd
# crm = pd.read_csv('data/industries.csv')
df = pd.read_csv('data/ads_insights.csv')

In [ ]:
q = 4
pd.qcut(df.loc[df.facebook_spend.notna(), 'facebook_spend'], q=q, labels=list(range(q))).isna().sum()

In [ ]:
df.industry.count()

In [ ]:
import pandas as pd
from enum import Enum
from itertools import combinations
from src.utils.enum import get_enum_values
import numpy as np

class EPlatform(str, Enum):
    facebook = "facebook"
    tiktok = "tiktok"
    google = "google"

df = pd.read_csv('data/ads_insights.csv')

In [ ]:
"-".join(['a', 'b'])

In [ ]:
set(combinations(, r=2))

In [ ]:
def get_all_subsets(s: set | list):
    n = len(s)
    subsets = []
    for i in range(n+1):
        i_subsets = combinations(s, i)
        subsets.extend(i_subsets)
    return subsets

In [ ]:
platforms = get_enum_values(EPlatform)
platform_combinations = get_all_subsets(platforms)

In [ ]:
table = pd.DataFrame(columns=['combination', 'shops', 'budget_split'])

In [ ]:
def get_spend_ratio(s: pd.Series, platforms: list):
    ratio = [s[f"{platform}_spend"] / s.full_spend for platform in platforms]
    return ratio

In [ ]:
def get_data_by_platform_combination(df: pd.DataFrame, combination: list):
    filter = [True in range(len(df))]
    for platform in platforms:
        if platform in combination:
            filter = filter & (df[f'{platform}_spend'].notna())
        else:
            filter = filter & (df[f'{platform}_spend'].isna())
    filtered = df[filter].copy()
    filtered['full_spend'] = filtered.apply(lambda df: sum([df[f"{platform}_spend"] for platform in combination]), axis=1)
    platforms_without_the_last_one = combination[:len(combination)-1]
    filtered['spend_ratio'] = filtered.apply(lambda df: get_spend_ratio(df, platforms=platforms_without_the_last_one), axis=1)
    mean_ratio = np.mean(filtered['spend_ratio'].tolist(), axis=0)
    print(mean_ratio)
    result = {"combination": '_'.join(combination), "shops": len(filtered)/len(df), "budget_split": list(mean_ratio) + [1-mean_ratio.sum()]}
    return result

In [ ]:
for combination in platform_combinations:
    if not len(combination):
        continue
    row = get_data_by_platform_combination(df=df, combination=combination)
    table.loc[len(table), :] = row

In [ ]:
table

In [ ]:
result

In [ ]:
filtered

In [ ]:
filtered.spend_ratio

In [ ]:
filtered

In [ ]:
combination[:len(combination)-1]

In [ ]:
len(df[(df.facebook_spend.notna()) & (df.google_spend.isna()) & (df.tiktok_spend.notna())])

In [ ]:
df = ping_ads_insights_all_platforms(db=db, columns='spend')

In [ ]:
only_shop = df.groupby('shop_id').sum().reset_index()

In [ ]:
df

In [ ]:
FacebookAd.ad_id.__mapper__

In [ ]:
FacebookAd.__class__

In [ ]:
df = ping_shops(db=db)

In [ ]:
df.loc[df.shop_id == 16038, :]

In [ ]:
df.ad_id

In [ ]:
df.loc[df.ad_id == '10'].index[0]

In [ ]:
query = session.query(FacebookAd.ad_id)

In [ ]:
query[0]

In [ ]:
type(query)